# setup

In [3]:
import sys
sys.path.append('../../..')

In [4]:
from copulae.input import generate_copula_net_input
from copulae.training.cflax.binorm import PositiveBiNormalCopula
from copulae.training.cflax.mono_aux import PositiveLayer
from copulae.training.cflax.mono_aux import EluPOne

from copulae.training.loss import sq_error, sq_error_partial, copula_likelihood
from copulae.training import setup_training

In [5]:
import jax.numpy as jnp
import jax.scipy.stats as jss
import jax

from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import optax
import pickle
from scipy.stats import bootstrap

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# data
https://github.com/yutingng/gen-AC.git

In [7]:
import pandas as pd
import scipy
import copy

from sklearn.model_selection import train_test_split

In [8]:
def add_train_random_noise(data, num_adds):
  new_data = np.random.rand(num_adds, data.shape[1])
  return np.concatenate((data, new_data), axis = 0)

In [9]:
def rank_normalization(X):
  X = copy.deepcopy(X)
  for z in X:
      ndata = z.shape[0]
      gap = 1./(ndata+1)
      nfeats = z.shape[1]
      for i in range(nfeats):
          z[:, i] = scipy.stats.rankdata(z[:, i], 'ordinal')*gap
  return X

In [10]:
!git clone https://github.com/yutingng/gen-AC.git

Cloning into 'gen-AC'...
remote: Enumerating objects: 466, done.
remote: Counting objects: 100% (466/466), done.
remote: Compressing objects: 100% (339/339), done.
remote: Total 466 (delta 159), reused 421 (delta 123), pack-reused 0
Receiving objects: 100% (466/466), 10.28 MiB | 8.49 MiB/s, done.
Resolving deltas: 100% (159/159), done.


In [11]:
class Boston():
  def __init__(self):
    # read
    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep = "\s+", skiprows = 22, header = None)
    X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    y = raw_df.values[1::2, 2]

    # split
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, random_state = 142857)
    X_train = np.concatenate((X_train, y_train[:, None]), axis = 1)
    X_test  = np.concatenate((X_test, y_test[:, None]), axis = 1)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # noise
    X_train = add_train_random_noise(X_train, int(X_train.shape[0]*0.01))

    # 2d
    train_data = X_train[:, [0, 13]]
    test_data = X_test[:, [0, 13]]

    # flip
    train_data[:, 0] = 1 - train_data[:, 0]
    test_data[:, 0] = 1 - test_data[:, 0]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [12]:
class INTC_MSFT():
  def __init__(self):
    # read
    intel_f = open('gen-AC/data/raw/INTC_MSFT_GE/INTEL.data', 'r')
    intel = np.array(list(map(float, intel_f.readlines())))

    ms_f = open('gen-AC/data/raw/INTC_MSFT_GE/MS.data', 'r')
    ms = np.array(list(map(float, ms_f.readlines())))

    ge_f = open('gen-AC/data/raw/INTC_MSFT_GE/GE.data', 'r')
    ge = np.array(list(map(float, ge_f.readlines())))

    # split
    X = np.concatenate((intel[:, None], ms[:, None]), axis = 1)
    X_train, X_test, _, _ = train_test_split(X, X, shuffle = True, random_state = 142857)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # 2d, noise
    train_data = X_train[:, [0, 1]]
    train_data = add_train_random_noise(train_data, int(train_data.shape[0]*0.01))
    test_data = X_test[:, [0, 1]]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [13]:
class GOOG_FB():
  def __init__(self):
    # read
    goog_f = open('gen-AC/data/raw/FB_GOOG/goog/close.vals', 'r')
    goog = np.array(list(map(float, goog_f.readlines())))

    fb_f = open('gen-AC/data/raw/FB_GOOG/fb/close.vals', 'r')
    fb = np.array(list(map(float, fb_f.readlines())))

    # split
    X = np.concatenate((goog[:, None], fb[:, None]), axis = 1)
    X_train, X_test, _, _ = train_test_split(X, X, shuffle=True, random_state=142857)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # 2d, noise
    train_data = X_train[:, [0, 1]]
    train_data = add_train_random_noise(train_data, int(train_data.shape[0]*0.01))
    test_data = X_test[:, [0, 1]]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [14]:
def get_set(D_val, data_points):
  points = D_val
  points = jnp.expand_dims(points, axis=0)

  # PDF and CDF for X
  kde_x = jss.gaussian_kde(data_points[0], bw_method='silverman')
  density_x = kde_x.evaluate(points[0, 0, :])
  cumulative_x = jnp.array([kde_x.integrate_box_1d(-jnp.inf, p) for p in points[0, 0, :]])

  # PDF and CDF for Y
  kde_y = jss.gaussian_kde(D[1], bw_method='silverman')
  density_y = kde_y.evaluate(points[0, 1, :])
  cumulative_y = jnp.array([kde_y.integrate_box_1d(-jnp.inf, p) for p in points[0, 1, :]])

  I_pdf = density_x.T * density_y.T
  I_pdf = jnp.expand_dims(I_pdf, axis=0)
  cdf_xy = jnp.array((cumulative_x, cumulative_y))
  cdf_xy = jnp.expand_dims(cdf_xy, axis=0)

  del density_x
  del density_y
  del cumulative_x
  del cumulative_y

  return points, I_pdf, cdf_xy

In [15]:
np.random.seed(30091985)
key = jax.random.PRNGKey(30091985)

# params

In [16]:
layer_widths = [128, 64, 32, 16]
losses = [
    (0.01, sq_error),
    (0.5, sq_error_partial),
    (0.1, copula_likelihood),
]
lr = 2e-3
n_iter = 10000
label = '2catsng'

In [17]:
def L_d(losses, params, state):
  loss = jnp.zeros((1,len(losses)), dtype=jnp.float32)
  for i, (w, loss_func) in enumerate(losses):
    loss = loss.at[0, i].set(w * loss_func(params, state))
  return loss

losses_eval = [
    (1.0, sq_error),
    (1.0, sq_error_partial),
    (1.0, copula_likelihood),
]

# boston

In [18]:
model = PositiveBiNormalCopula(
    PositiveLayer(layer_widths, EluPOne, EluPOne, EluPOne)
)

In [19]:
data_loader = Boston()
D = np.array([data_loader.train_x, data_loader.train_y])[:, :, 0]
TrainingTensors = generate_copula_net_input(
    D=D,
    bootstrap=False
)

In [20]:
nn_C, nn_dC, nn_c, cop_state, forward, grad = setup_training(
    model, TrainingTensors, losses
)

In [21]:
_, subkey = jax.random.split(key)
init_params = model.init(subkey, TrainingTensors.UV_batches[0])
del subkey

params = init_params
optimizer = optax.adam(lr)
opt_state = optimizer.init(params)

In [22]:
best = 1e6
for i in tqdm(range(n_iter)):
    grads, cop_state = grad(params, cop_state)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        loss = L_d(losses_eval, params, cop_state)
        if loss[0][-1] < best:
          best_params = params
          best_cop_state = cop_state
          best = loss[0][-1]
        print('Iter {}. Loss {}'.format(i, loss))

  0%|          | 1/10000 [01:19<219:43:02, 79.11s/it]

Iter 0. Loss [[0.39889488 0.22860757 9.521027  ]]


  0%|          | 12/10000 [02:22<3:19:51,  1.20s/it]

Iter 10. Loss [[0.37027937 0.13897829 4.493246  ]]


  0%|          | 22/10000 [02:25<41:39,  3.99it/s]

...

100%|█████████▉| 9981/10000 [45:18<00:06,  2.93it/s]

Iter 9980. Loss [[ 0.05328202  0.29531145 19.931568  ]]


100%|█████████▉| 9991/10000 [45:22<00:04,  2.17it/s]

Iter 9990. Loss [[ 0.05328149  0.29531145 19.931568  ]]


100%|██████████| 10000/10000 [45:25<00:00,  3.67it/s]


In [23]:
density_graph_points, I_pdf, cdf_xy = get_set(D, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))
np.mean(yhat)

0.5001761

In [24]:
D_val = np.array([data_loader.validation_x, data_loader.validation_y])[:, :, 0]
density_graph_points, I_pdf, cdf_xy = get_set(D_val, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))

L_d(losses_eval, best_params, best_cop_state), np.mean(yhat)

(Array([[0.12206782, 0.03574042, 0.54431593]], dtype=float32), 0.50437844)

In [25]:
res = bootstrap(yhat, np.mean)
res.standard_error, res.confidence_interval

(0.118768856,
 ConfidenceInterval(low=0.35409185489304135, high=0.9342322954372339))

In [26]:
with open('drive/MyDrive/copulae/models/{}_boston.pkl'.format(label), 'wb') as fp:
    pickle.dump(best_params, fp)

# intc-msft

In [27]:
model = PositiveBiNormalCopula(
    PositiveLayer(layer_widths, EluPOne, EluPOne, EluPOne)
)

In [28]:
data_loader = INTC_MSFT()
D = np.array([data_loader.train_x, data_loader.train_y])[:, :, 0]
TrainingTensors = generate_copula_net_input(
    D=D,
    bootstrap=False
)

In [29]:
nn_C, nn_dC, nn_c, cop_state, forward, grad = setup_training(
    model, TrainingTensors, losses
)

In [30]:
_, subkey = jax.random.split(key)
init_params = model.init(subkey, TrainingTensors.UV_batches[0])
del subkey

params = init_params
optimizer = optax.adam(lr)
opt_state = optimizer.init(params)

In [31]:
best = 1e6
for i in tqdm(range(n_iter)):
    grads, cop_state = grad(params, cop_state)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        loss = L_d(losses_eval, params, cop_state)
        if loss[0][-1] < best:
          best_params = params
          best_cop_state = cop_state
          best = loss[0][-1]
        print('Iter {}. Loss {}'.format(i, loss))

  0%|          | 1/10000 [01:16<212:04:22, 76.35s/it]

Iter 0. Loss [[0.40008873 0.24309562 8.6101    ]]


  0%|          | 11/10000 [02:14<4:20:13,  1.56s/it]

Iter 10. Loss [[0.38002777 0.1390463  2.3202221 ]]


  0%|          | 21/10000 [02:17<56:10,  2.96it/s]  

...

100%|█████████▉| 9981/10000 [51:19<00:05,  3.48it/s]

Iter 9980. Loss [[ 0.1927381   0.29995158 19.931568  ]]


100%|█████████▉| 9991/10000 [51:21<00:02,  4.41it/s]

Iter 9990. Loss [[ 0.19273806  0.29995158 19.931568  ]]


100%|██████████| 10000/10000 [51:24<00:00,  3.24it/s]


In [32]:
density_graph_points, I_pdf, cdf_xy = get_set(D, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))
np.mean(yhat)

0.1637681

In [33]:
D_val = np.array([data_loader.validation_x, data_loader.validation_y])[:, :, 0]
density_graph_points, I_pdf, cdf_xy = get_set(D_val, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))

L_d(losses_eval, best_params, best_cop_state), np.mean(yhat)

(Array([[0.36910436, 0.00451887, 0.08953402]], dtype=float32), 0.22382924)

In [34]:
res = bootstrap(yhat, np.mean)
res.standard_error, res.confidence_interval

(0.043124724,
 ConfidenceInterval(low=0.14633735498395775, high=0.31634542327632625))

In [35]:
with open('drive/MyDrive/copulae/models/{}_intcmsft.pkl'.format(label), 'wb') as fp:
    pickle.dump(best_params, fp)

# goog-fb

In [36]:
model = PositiveBiNormalCopula(
    PositiveLayer(layer_widths, EluPOne, EluPOne, EluPOne)
)

In [37]:
data_loader = GOOG_FB()
D = np.array([data_loader.train_x, data_loader.train_y])[:, :, 0]
TrainingTensors = generate_copula_net_input(
    D=D,
    bootstrap=False
)

In [38]:
nn_C, nn_dC, nn_c, cop_state, forward, grad = setup_training(
    model, TrainingTensors, losses
)

In [39]:
_, subkey = jax.random.split(key)
init_params = model.init(subkey, TrainingTensors.UV_batches[0])
del subkey

params = init_params
optimizer = optax.adam(lr)
opt_state = optimizer.init(params)

In [40]:
best = 1e6
for i in tqdm(range(n_iter)):
    grads, cop_state = grad(params, cop_state)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        loss = L_d(losses_eval, params, cop_state)
        if loss[0][-1] < best:
          best_params = params
          best_cop_state = cop_state
          best = loss[0][-1]
        print('Iter {}. Loss {}'.format(i, loss))

  0%|          | 1/10000 [01:13<204:52:07, 73.76s/it]

Iter 0. Loss [[0.32449996 0.1930944  8.615612  ]]


  0%|          | 11/10000 [02:13<4:19:27,  1.56s/it]

Iter 10. Loss [[0.31494874 0.14555949 1.4112115 ]]


  0%|          | 21/10000 [02:15<41:46,  3.98it/s]

...

100%|█████████▉| 9981/10000 [49:41<00:05,  3.37it/s]

Iter 9980. Loss [[ 0.36175013  0.0622369  -1.5961312 ]]


100%|█████████▉| 9991/10000 [49:43<00:01,  4.60it/s]

Iter 9990. Loss [[ 0.3618585   0.06283072 -1.6019098 ]]


100%|██████████| 10000/10000 [49:45<00:00,  3.35it/s]


In [41]:
density_graph_points, I_pdf, cdf_xy = get_set(D, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))
np.mean(yhat)

-0.9725211

In [42]:
D_val = np.array([data_loader.validation_x, data_loader.validation_y])[:, :, 0]
density_graph_points, I_pdf, cdf_xy = get_set(D_val, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))

L_d(losses_eval, best_params, best_cop_state), np.mean(yhat)

(Array([[ 0.34629548,  0.12875281, -1.8662301 ]], dtype=float32), -1.0099615)

In [43]:
res = bootstrap(yhat, np.mean)
res.standard_error, res.confidence_interval

(0.034368455,
 ConfidenceInterval(low=-1.0747679914654162, high=-0.9392954429064866))

In [44]:
with open('drive/MyDrive/copulae/models/{}_googfb.pkl'.format(label), 'wb') as fp:
    pickle.dump(best_params, fp)